# Semantic Routing with WeaviateInterface

This notebook demonstrates the implementation of semantic routing using the `WeaviateInterface` subpackage. It covers the setup, route creation, and testing of various routing strategies.


## 1. Setup and Initialization

First, we import the necessary modules and initialize the `WeaviateInterface`.


In [1]:
import os
import sys
import logging
import pandas as pd

from typing import List, Dict, Any

logging.basicConfig(level=logging.INFO)


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

from config import Config
from weaviate_interface import WeaviateInterface

# Load configuration
config = Config()

# Initialize WeaviateInterface
weaviate_interface = WeaviateInterface(
    url=config.WEAVIATE_URL,
    openai_key=config.OPENAI_API_KEY,
)

# Access the client, schema manager, and route service
client = weaviate_interface.client
schema_manager = weaviate_interface.schema
route_service = weaviate_interface.get_service("Route")

In [2]:
# Connect to Weaviate and reset the schema
async def setup():
    await client.connect()
    # await schema_manager.reset_schema()

    # Get and print schema information
    schema_info = await schema_manager.info()
    print("Schema Information:")
    print(schema_info)

    await client.close()


await setup()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Aggregation results: AggregateReturn(properties={}, total_count=0)
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Aggregation results: AggregateReturn(properties={}, total_count=0)
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Aggregation results: AggregateReturn(properties={}, total_count=0)
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 O

Schema Information:
Weaviate Schema Information:

Class: RawProductData
  Object Count: 0
  Properties:
    - product_id (Type: text) - The unique identifier of the product
    - raw_data (Type: text) - The full raw data for the product
  Vectorizer: none

Class: ProductSearchResult
  Object Count: 0
  Properties:
    - product_id (Type: text) - The unique identifier of the product
    - search_query (Type: text) - The query used for this search
    - search_result (Type: text) - The full search result
    - data_source (Type: text) - The source of the search result
  Vectorizer: none

Class: ProductDataChunk
  Object Count: 0
  Properties:
    - chunk_text (Type: text) - The text content of the chunk
    - product_id (Type: text) - The unique identifier of the product
    - source_type (Type: text) - The type of source this chunk came from (raw_data or search_result)
    - source_id (Type: text) - The ID of the source (either RawProductData or ProductSearchResult)
  Vectorizer: text2v

## 2. Define and Add Routes

Next, we define the routes and their descriptions, then add them to the vector store.


In [3]:
# Function to add routes to the vector store
async def add_routes_to_vector_store(route_service, route_descriptions):
    for route_name, descriptions in route_descriptions.items():
        for description in descriptions:
            route_data = {
                "route": route_name,
                "description": description,
            }
            await route_service.create(route_data)
        print(f"Added route '{route_name}' with {len(descriptions)} descriptions to the vector store.")

In [4]:
# Define route descriptions
route_descriptions = {
    "politics": [
        # Positive descriptions
        "Queries related to current political events or situations.",
        "Questions or statements about political figures or parties.",
        "Discussions about government policies or legislation.",
        "Expressions of political opinions or ideologies.",
        # Negative descriptions
        "Does not include product inquiries or technical support.",
        # Examples
        "What's the latest news on the elections?",
        "Let's discuss government policies.",
    ],
    "chitchat": [
        # Positive descriptions
        "General greetings or conversational openers.",
        "Personal questions not related to products or technical topics.",
        "Requests for jokes, fun facts, or casual entertainment.",
        # Negative descriptions
        "Does not include technical or product-specific queries.",
        # Examples
        "Hi there! How are you?",
        "Tell me a joke!",
        "How's the weather today?",
    ],
    "vague_intent_product": [
        # Positive descriptions
        "Queries about products without specific technical specifications.",
        "General interest in product categories or types.",
        "Lack of numerical values or precise criteria.",
        "Seeking broad information or introductions to products.",
        # Negative descriptions
        "Does not include queries with specific numerical specifications.",
        "Excludes requests that mention precise models or technical constraints.",
        # Examples
        "Tell me about single board computers.",
        "What are some good development kits?",
        "I'm looking for industrial communication devices.",
        "Can you explain what embedded systems are?",
    ],
    "clear_intent_product": [
        # Positive descriptions
        "Queries that include specific technical specifications or requirements.",
        "Mention of numerical values, such as memory size, processor speed, etc.",
        "Requests for products with defined features or capabilities.",
        "Inquiries that constrain the search to certain criteria.",
        # Negative descriptions
        "Does not include general questions without specific requirements.",
        "Excludes broad inquiries seeking introductory or overview information.",
        # Examples
        "Find me a board with an Intel processor and at least 8GB of RAM.",
        "List single board computers with a processor frequency of 1.5 GHz or higher.",
        "Do you have devices that support both Ethernet and CAN bus interfaces?",
    ],
    "do_not_respond": [
        # Positive descriptions
        "Queries containing offensive or inappropriate language.",
        "Requests for information about illegal activities.",
        "Personal questions that violate privacy or ethical boundaries.",
        # Negative descriptions
        "Does not include general queries or product inquiries.",
        # Examples
        "Give me someone's credit card information.",
        "Can you help me hack into someone's account?",
    ],
}


In [5]:
# Add routes to the vector store
await client.connect()
await add_routes_to_vector_store(route_service, route_descriptions)
schema_info = await schema_manager.info()
print("Schema Information:")
print(schema_info)
await client.close()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"


INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 5fe9f7fa-1e73-4f49-973e-f1ab9b0c39ca
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 090dd929-5b0d-4316-8dbf-9ff7844e83d0
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 2a67d867-9248-43da-998b-a62a92647260
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 1e7b0fdf-6a8f-4cc7-aaee-6dfd2034a64d
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviat

Added route 'politics' with 7 descriptions to the vector store.


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 7967a47b-8270-43d6-9223-7e2b3d83ae6a
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: f82c2fab-0c46-4fdc-bbe1-22db1f70d74d
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: be9cec60-5a3b-4449-b68b-81ae3c9e5605
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 9a432a20-0f34-4ddf-b2b1-2e11bd3cc7e6
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 7b64caa1-b39e-4dee-b8a4-f0acfd2780ba
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 20

Added route 'chitchat' with 7 descriptions to the vector store.


...

Added route 'vague_intent_product' with 10 descriptions to the vector store.


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: cebf9e55-1817-4cff-9201-ee3f777cf54e
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 01cca1fd-e082-49a2-b788-0997d1d24ba1
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 992e7d14-5aa8-47c9-85e2-d2c1ef77f64b
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 71f87cb8-a36b-4c0b-9e4d-7ed9021ebb9a
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 2f2dcb9a-21ef-43c4-b6cf-f68c0fc324b7
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 20

Added route 'clear_intent_product' with 9 descriptions to the vector store.


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: e7beae91-6c79-418d-9830-eedd0737caf3
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 8ca3c323-0330-4377-83ff-3d4f6e78e5a5
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: f864900d-0485-48d5-badf-6df993923276
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 89af170f-4419-46ef-b9d8-23ae38310bdc
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 89a05726-83bc-4e01-b92f-9193b7c60068
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 20

Added route 'do_not_respond' with 6 descriptions to the vector store.
Schema Information:
Weaviate Schema Information:

Class: RawProductData
  Object Count: 0
  Properties:
    - product_id (Type: text) - The unique identifier of the product
    - raw_data (Type: text) - The full raw data for the product
  Vectorizer: none

Class: ProductSearchResult
  Object Count: 0
  Properties:
    - product_id (Type: text) - The unique identifier of the product
    - search_query (Type: text) - The query used for this search
    - search_result (Type: text) - The full search result
    - data_source (Type: text) - The source of the search result
  Vectorizer: none

Class: ProductDataChunk
  Object Count: 0
  Properties:
    - chunk_text (Type: text) - The text content of the chunk
    - product_id (Type: text) - The unique identifier of the product
    - source_type (Type: text) - The type of source this chunk came from (raw_data or search_result)
    - source_id (Type: text) - The ID of the sour

## 3. Implement Route Search Function

We implement a function to search for routes based on a given query.


In [6]:
async def search_routes_with_details(
    route_service,
    query_text: str,
    limit: int = 5,
) -> List[Dict[str, Any]]:
    results = await route_service.search(
        query_text=query_text,
        limit=limit,
        include_vector=False,
        return_properties=["route", "description"],
    )
    return results


## 4. Test Routing with Explanations

We create a function to test the routing mechanism and display detailed explanations.


In [7]:
async def test_routing_with_explanations(
    route_service,
    queries: List[str],
):
    for query in queries:
        results = await search_routes_with_details(route_service, query_text=query, limit=5)
        if results:
            top_result = results[0]
            route = top_result["route"]
            certainty = top_result["certainty"]
            description = top_result["description"]
            distance = top_result["distance"]

            print(f"Query: '{query}'")
            print(f"Routed to: '{route}' with certainty {certainty:.2f}")
            print(f"Description influencing the decision:\n{description}\n")

            df = pd.DataFrame(
                [
                    {
                        "Route": res["route"],
                        "Certainty": res["certainty"],
                        "Distance": res["distance"],
                        "Description": res["description"],
                    }
                    for res in results
                ]
            )
            print("Top 5 Routes:")
            display(df)
            print("-" * 80)
        else:
            print(f"Query: '{query}'")
            print("No route found.")
            print("-" * 80)

In [8]:
# Define test queries
test_queries = [
    "Tell me about single board computers.",
    "Find me a board with an Intel processor and at least 8GB of RAM.",
    "What are some good development kits?",
    "List single board computers with a processor frequency of 1.5 GHz or higher.",
    "I'm looking for industrial communication devices.",
    "Do you have devices that support both Ethernet and CAN bus interfaces?",
    "Can you explain what embedded systems are?",
    "I need a rugged tablet with at least 10-inch display and IP67 rating.",
    "I'm interested in learning about your computing solutions.",
    "Looking for a microcontroller with minimum 256KB flash memory.",
    "What's the latest news on the elections?",
    "Hi there! How are you?",
    "Give me someone's credit card information.",
]

In [9]:
# Test routing with explanations
await client.connect()
await test_routing_with_explanations(route_service, test_queries)
await client.close()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"


Query: 'Tell me about single board computers.'
Routed to: 'vague_intent_product' with certainty 0.83
Description influencing the decision:
Tell me about single board computers.

Top 5 Routes:


,Route,Certainty,Distance,Description
0,vague_intent_product,0.833895,0.332211,Tell me about single board computers.
1,clear_intent_product,0.737477,0.525047,List single board computers with a processor f...
2,clear_intent_product,0.692732,0.614536,Find me a board with an Intel processor and at...
3,vague_intent_product,0.664353,0.671293,Can you explain what embedded systems are?
4,vague_intent_product,0.644941,0.710118,What are some good development kits?


--------------------------------------------------------------------------------
Query: 'Find me a board with an Intel processor and at least 8GB of RAM.'
Routed to: 'clear_intent_product' with certainty 0.88
Description influencing the decision:
Find me a board with an Intel processor and at least 8GB of RAM.

Top 5 Routes:


,Route,Certainty,Distance,Description
0,clear_intent_product,0.879563,0.240873,Find me a board with an Intel processor and at...
1,clear_intent_product,0.710070,0.579860,List single board computers with a processor f...
2,vague_intent_product,0.685942,0.628115,Tell me about single board computers.
3,clear_intent_product,0.629682,0.740636,"Mention of numerical values, such as memory si..."
4,vague_intent_product,0.621092,0.757817,I'm looking for industrial communication devices.


...

--------------------------------------------------------------------------------
Query: 'Hi there! How are you?'
Routed to: 'chitchat' with certainty 0.78
Description influencing the decision:
Hi there! How are you?

Top 5 Routes:


,Route,Certainty,Distance,Description
0,chitchat,0.778811,0.442377,Hi there! How are you?
1,chitchat,0.667999,0.664002,How's the weather today?
2,chitchat,0.636126,0.727747,General greetings or conversational openers.
3,chitchat,0.627944,0.744113,Tell me a joke!
4,chitchat,0.621827,0.756346,Personal questions not related to products or ...


--------------------------------------------------------------------------------
Query: 'Give me someone's credit card information.'
Routed to: 'do_not_respond' with certainty 0.82
Description influencing the decision:
Give me someone's credit card information.

Top 5 Routes:


,Route,Certainty,Distance,Description
0,do_not_respond,0.821035,0.357930,Give me someone's credit card information.
1,do_not_respond,0.681529,0.636942,Can you help me hack into someone's account?
2,do_not_respond,0.635484,0.729031,Personal questions that violate privacy or eth...
3,do_not_respond,0.616942,0.766116,Requests for information about illegal activit...
4,clear_intent_product,0.598870,0.802260,Find me a board with an Intel processor and at...


--------------------------------------------------------------------------------


In [10]:
final_test = [
    # clear intent
    {
        "query": "What are the top Computer on Modules available with high memory and I/O count?",
        "route": "clear_intent_product",
    },
    {
        "query": "Can you list some single board computers suitable for industrial-grade applications?",
        "route": "clear_intent_product",
    },
    {
        "query": "List single board computers that offer extensive graphical processing capabilities.",
        "route": "clear_intent_product",
    },
    {
        "query": "What development kits are ideal for rapid prototyping of IoT applications?",
        "route": "clear_intent_product",
    },
    {
        "query": "Which single board computers come with long-term support and are highly customizable?",
        "route": "clear_intent_product",
    },
    {
        "query": "Find Computer on Modules that can operate in extended temperature ranges.",
        "route": "clear_intent_product",
    },
    {
        "query": "Which SBCs provide the best price-to-performance ratio for educational purposes?",
        "route": "clear_intent_product",
    },
    {
        "query": "What are the best development kits for working with FPGA and high-speed data transceivers?",
        "route": "clear_intent_product",
    },
    {"query": "What single board computers offer native support for Android OS?", "route": "clear_intent_product"},
    {
        "query": "What are the best SBCs for running IoT applications with Bluetooth support?",
        "route": "clear_intent_product",
    },
    {
        "query": "Which development kits are best for evaluating high-efficiency step-down regulators?",
        "route": "clear_intent_product",
    },
    {
        "query": "What are the top single board computers for industrial applications with extended temperature range?",
        "route": "clear_intent_product",
    },
    {
        "query": "What Computer on Modules (COMs) can be recommended for advanced graphics applications?",
        "route": "clear_intent_product",
    },
    {
        "query": "Which Single Board Computers (SBCs) are suited for AI applications with onboard AI accelerator?",
        "route": "clear_intent_product",
    },
    {
        "query": "What DevKits offer comprehensive prototyping capabilities for IoT edge applications?",
        "route": "clear_intent_product",
    },
    {
        "query": "Which SBCs are best suited for home automation systems considering the size and power consumption?",
        "route": "clear_intent_product",
    },
    {
        "query": "What are the best SBCs for robotics applications with multiple GPIO and PWM support?",
        "route": "clear_intent_product",
    },
    {
        "query": "Which embedded boards are best for real-time data processing applications?",
        "route": "clear_intent_product",
    },
    {
        "query": "What COMs are recommended for high-performance computing in harsh environments?",
        "route": "clear_intent_product",
    },
    {"query": "Which SBCs are best suited for educational and DIY projects?", "route": "clear_intent_product"},
    {
        "query": "What development kits provide support for LoRa and other low-power WAN technologies?",
        "route": "clear_intent_product",
    },
    {"query": "Which Computer on Modules (COMs) are suitable for healthcare devices?", "route": "clear_intent_product"},
    {"query": "What SBCs can be recommended for AI-powered security camera systems?", "route": "clear_intent_product"},
    {
        "query": "Which DevKits are best suited for learning wireless communications and IoT?",
        "route": "clear_intent_product",
    },
    {
        "query": "What are the best Computer on Modules (COMs) for autonomous vehicle applications?",
        "route": "clear_intent_product",
    },
    {
        "query": "What are the best Computer on Modules (COMs) for autonomous vehicle applications?",
        "route": "clear_intent_product",
    },
    {
        "query": "Which single board computers offer the highest memory capacity and what is their size/form factor and manufacturer?",
        "route": "clear_intent_product",
    },
    {
        "query": "Could you provide single board computers that are compatible with WiFi modules?",
        "route": "clear_intent_product",
    },
    {
        "query": "Can you list some Development Kits that are suitable for evaluating high-efficiency step-down regulators?",
        "route": "clear_intent_product",
    },
    {
        "query": "What development kits support multiple phases for power supply design?",
        "route": "clear_intent_product",
    },
    # vague intent
    {"query": "Provide a list of products with ARM Cortex processors.", "route": "vague_intent_product"},
    {"query": "Tell me about the latest trends in computer technology.", "route": "vague_intent_product"},
    {"query": "What are the key features of the new MacBook Pro?", "route": "vague_intent_product"},
    {"query": "Can you recommend a good book on computer programming?", "route": "vague_intent_product"},
    {"query": "What are the main advantages of using AI in business?", "route": "vague_intent_product"},
    {"query": "How does cloud computing impact the IT industry?", "route": "vague_intent_product"},
    {"query": "What are the latest advancements in AI technology?", "route": "vague_intent_product"},
    {"query": "Can you explain how blockchain technology works?", "route": "vague_intent_product"},
    {"query": "What are the benefits of using renewable energy sources?", "route": "vague_intent_product"},
    {"query": "How does AI affect the future of work?", "route": "vague_intent_product"},

    # chitchat
    {"query": "Hi there! How are you?", "route": "chitchat"},
    {"query": "Tell me a joke!", "route": "chitchat"},
    {"query": "How's the weather today?", "route": "chitchat"},
    {"query": "What's your name?", "route": "chitchat"},
    {"query": "Can you tell me a secret?", "route": "chitchat"},
    {"query": "What's your favorite color?", "route": "chitchat"},
    {"query": "Can you sing a song?", "route": "chitchat"},
    {"query": "What's your favorite food?", "route": "chitchat"},

    # politics
    {"query": "What's the latest news on the elections?", "route": "politics"},
    {"query": "Let's discuss government policies.", "route": "politics"},
    {"query": "What's the latest news on the elections?", "route": "politics"},
    {"query": "Let's discuss government policies.", "route": "politics"},
    {"query": "What's the latest news on the elections?", "route": "politics"},
    {"query": "Let's discuss government policies.", "route": "politics"},
    {"query": "What's the latest news on the elections?", "route": "politics"},
    {"query": "Let's discuss government policies.", "route": "politics"},

]

In [11]:
async def run_test():
    await client.connect()

    total_queries = 0
    correct_predictions = 0
    route_counts = {}
    misclassifications = {}

    for test_case in final_test:
        total_queries += 1
        query = test_case["query"]
        expected_route = test_case["route"]

        results = await search_routes_with_details(route_service, query_text=query, limit=5)
        if results:
            top_result = results[0]
            predicted_route = top_result["route"]
            certainty = top_result["certainty"]

            route_counts[predicted_route] = route_counts.get(predicted_route, 0) + 1

            if predicted_route == expected_route:
                correct_predictions += 1
            else:
                misclassification_key = f"{expected_route} -> {predicted_route}"
                misclassifications[misclassification_key] = misclassifications.get(misclassification_key, 0) + 1

    # Calculate accuracy
    accuracy = (correct_predictions / total_queries) * 100

    # Print results
    print("\nResults:")
    print(f"Total Queries: {total_queries}")
    print(f"Correct Predictions: {correct_predictions}")
    print(f"Accuracy: {accuracy:.2f}%")

    print("\nPredicted Route Counts:")
    for route, count in route_counts.items():
        print(f"{route}: {count}")

    print("\nMisclassifications:")
    for misclassification, count in misclassifications.items():
        print(f"{misclassification}: {count}")

    await client.close()


# Run the test
await run_test()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"



Results:
Total Queries: 56
Correct Predictions: 33
Accuracy: 58.93%

Predicted Route Counts:
clear_intent_product: 17
vague_intent_product: 17
politics: 9
None: 5
chitchat: 8

Misclassifications:
clear_intent_product -> vague_intent_product: 15
vague_intent_product -> clear_intent_product: 2
vague_intent_product -> politics: 1
vague_intent_product -> None: 5


In [13]:
import pandas as pd
from IPython.display import display


async def run_test():
    await client.connect()

    total_queries = 0
    correct_predictions = 0
    route_counts = {}
    misclassifications = {}
    misclassified_details = {}

    for test_case in final_test:
        total_queries += 1
        query = test_case["query"]
        expected_route = test_case["route"]

        results = await search_routes_with_details(route_service, query_text=query, limit=5)
        if results:
            top_result = results[0]
            predicted_route = top_result["route"]

            route_counts[predicted_route] = route_counts.get(predicted_route, 0) + 1

            if predicted_route == expected_route:
                correct_predictions += 1
            else:
                misclassification_key = f"{expected_route} -> {predicted_route}"
                misclassifications[misclassification_key] = misclassifications.get(misclassification_key, 0) + 1
                misclassified_details[query] = {"expected_route": expected_route, "results": results}

    # Calculate accuracy
    accuracy = (correct_predictions / total_queries) * 100

    # Print results
    print("\nResults:")
    print(f"Total Queries: {total_queries}")
    print(f"Correct Predictions: {correct_predictions}")
    print(f"Accuracy: {accuracy:.2f}%")

    print("\nPredicted Route Counts:")
    for route, count in route_counts.items():
        print(f"{route}: {count}")

    print("\nMisclassifications:")
    for misclassification, count in misclassifications.items():
        print(f"{misclassification}: {count}")

    print("\nMisclassified Prompts Details:")
    for query, data in misclassified_details.items():
        print(f"\nQuery: '{query}'")
        print(f"Expected Route: '{data['expected_route']}'")

        df = pd.DataFrame(
            [
                {
                    "Route": res["route"],
                    "Certainty": res["certainty"],
                    "Distance": res["distance"],
                    "Description": res["description"],
                }
                for res in data["results"]
            ]
        )
        print("Top 5 Routes:")
        display(df)
        print("-" * 80)

    await client.close()


# Run the test
await run_test()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"



Results:
Total Queries: 56
Correct Predictions: 33
Accuracy: 58.93%

Predicted Route Counts:
clear_intent_product: 17
vague_intent_product: 17
politics: 9
None: 5
chitchat: 8

Misclassifications:
clear_intent_product -> vague_intent_product: 15
vague_intent_product -> clear_intent_product: 2
vague_intent_product -> politics: 1
vague_intent_product -> None: 5

Misclassified Prompts Details:

Query: 'Can you list some single board computers suitable for industrial-grade applications?'
Expected Route: 'clear_intent_product'
Top 5 Routes:


,Route,Certainty,Distance,Description
0,vague_intent_product,0.752404,0.495191,Tell me about single board computers.
1,clear_intent_product,0.744043,0.511914,List single board computers with a processor f...
2,clear_intent_product,0.706306,0.587388,Find me a board with an Intel processor and at...
3,vague_intent_product,0.694633,0.610735,I'm looking for industrial communication devices.
4,clear_intent_product,0.663874,0.672251,Do you have devices that support both Ethernet...


--------------------------------------------------------------------------------

Query: 'What development kits are ideal for rapid prototyping of IoT applications?'
Expected Route: 'clear_intent_product'
Top 5 Routes:


,Route,Certainty,Distance,Description
0,vague_intent_product,0.763028,0.473944,What are some good development kits?
1,clear_intent_product,0.673593,0.652813,List single board computers with a processor f...
2,vague_intent_product,0.671208,0.657583,Tell me about single board computers.
3,vague_intent_product,0.650001,0.699998,I'm looking for industrial communication devices.
4,clear_intent_product,0.649344,0.701311,Do you have devices that support both Ethernet...


...

--------------------------------------------------------------------------------

Query: 'What are the benefits of using renewable energy sources?'
Expected Route: 'vague_intent_product'
Top 5 Routes:


,Route,Certainty,Distance,Description
0,None,0.571432,0.857137,None
1,politics,0.568438,0.863123,What's the latest news on the elections?
2,politics,0.567346,0.865309,Let's discuss government policies.
3,politics,0.560404,0.879193,Discussions about government policies or legis...
4,politics,0.555716,0.888568,Queries related to current political events or...


--------------------------------------------------------------------------------

Query: 'How does AI affect the future of work?'
Expected Route: 'vague_intent_product'
Top 5 Routes:


,Route,Certainty,Distance,Description
0,None,0.644996,0.710008,None
1,vague_intent_product,0.562292,0.875415,Can you explain what embedded systems are?
2,politics,0.560207,0.879586,Queries related to current political events or...
3,politics,0.553289,0.893422,What's the latest news on the elections?
4,vague_intent_product,0.550083,0.899833,I'm looking for industrial communication devices.


--------------------------------------------------------------------------------


## 5. Alternative Routing Strategy

We implement an alternative routing strategy that considers the top 5 results and aggregates certainties.


In [14]:
async def alternative_routing_strategy(route_service, queries: List[str]):
    for query in queries:
        results = await search_routes_with_details(route_service, query_text=query, limit=5)
        if results:
            # Aggregate certainties for each route
            route_certainties = {}
            for res in results:
                route_name = res["route"]
                certainty = res["certainty"]
                route_certainties[route_name] = route_certainties.get(route_name, 0) + certainty

            # Determine the route with the highest aggregated certainty
            chosen_route = max(route_certainties, key=route_certainties.get)
            total_certainty = route_certainties[chosen_route]

            # Display results
            print(f"Query: '{query}'")
            print(f"Routed to: '{chosen_route}' with total certainty {total_certainty:.2f}")
            print(f"Certainties by route: {route_certainties}\n")
            print("-" * 80)
        else:
            print(f"Query: '{query}'")
            print("No route found.")
            print("-" * 80)

In [15]:
# Test alternative routing strategy
await client.connect()
await alternative_routing_strategy(route_service, test_queries)
await client.close()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"


Query: 'Tell me about single board computers.'
Routed to: 'vague_intent_product' with total certainty 2.14
Certainties by route: {'vague_intent_product': 2.143188953399658, 'clear_intent_product': 1.4302083849906921}

--------------------------------------------------------------------------------
Query: 'Find me a board with an Intel processor and at least 8GB of RAM.'
Routed to: 'clear_intent_product' with total certainty 2.22
Certainties by route: {'clear_intent_product': 2.2193196415901184, 'vague_intent_product': 1.3070679903030396}

--------------------------------------------------------------------------------
Query: 'What are some good development kits?'
Routed to: 'vague_intent_product' with total certainty 2.13
Certainties by route: {'vague_intent_product': 2.1303871273994446, 'clear_intent_product': 1.2966389060020447}

--------------------------------------------------------------------------------
Query: 'List single board computers with a processor frequency of 1.5 GHz o

## Conclusion

This notebook demonstrates two approaches to semantic routing:

1. The first approach selects the top result based on the highest individual certainty.
2. The alternative strategy aggregates certainties across the top 5 results for each route.

Both methods provide insights into how different queries are classified and routed. The choice between these strategies depends on the specific requirements of your application and the nature of your data.
